In [24]:
#  -l --list-models    List supported models
#     --model          LLM model to use (default: openai/gpt-4)
#     --temperature    LLM model temperature (default: 0.8)
#     --top-p          LLM model top_p (default: 0.95)
#     --max-tokens     LLM model max_tokens (default: 1024)
#     --examples       Number of in-context examples to use (default: 0)
#     --lang           What language to generate (default: verilog)
#     --output         File to write extracted code

In [25]:
import argparse
import sys
import os
import re
import time
import getpass
import keys

from langchain_openai import ChatOpenAI
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_community.chat_models import ChatAnyscale
from langchain.schema import SystemMessage, HumanMessage
from langchain_community.callbacks import get_openai_callback

In [26]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]= keys.LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"]="pr-stupendous-reveal-16"

os.environ["OPENAI_API_KEY"] = keys.OPENAI_API_KEY

In [27]:
llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-06a5b79a-cca9-49f4-b802-4f13c820083d-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [28]:
#-------------------------------------------------------------------------
# Models
#-------------------------------------------------------------------------

openai_chat_models = [
  "gpt-4",
]

nim_chat_models = [
]

anyscale_chat_models = [
]

all_chat_models = [
  *openai_chat_models,
  *nim_chat_models,
  *anyscale_chat_models,
]

model_aliases = {
  "openai/gpt4"            : "gpt-4",
}

In [29]:
#-------------------------------------------------------------------------
# Context
#-------------------------------------------------------------------------

verilog_system_msg="""
You are a Verilog RTL designer that only writes code using correct
Verilog syntax.
"""

pymtl_system_msg="""
You are a PyMTL3 RTL designer that only writes code using correct
PyMTL3 syntax.
"""

pyrtl_system_msg="""
You are a PyRTL RTL designer that only writes code using correct
PyRTL syntax.
"""

myhdl_system_msg="""
You are a MyHDL RTL designer that only writes code using correct
MyHDL syntax.
"""

migen_system_msg="""
You are a Migen RTL designer that only writes code using correct
Migen syntax.
"""

amaranth_system_msg="""
You are a Amaranth RTL designer that only writes code using correct
Amaranth syntax.
"""

prompt_no_explain_suffix="""
Enclose your code with <CODE> and </CODE>. Only output the code snippet
and do NOT output anything else.
"""

In [30]:
model =           "gpt-4"
problem =         "Prob01p01_comb_const_zero"
temperature =     0.8
top_p =           0.95
max_tokens =      1024
examples =        0 # the code does not load any examples
lang =            "verilog"
generate_log =    f"output/{problem}_generate.txt"
output =          f"output/{problem}_output.v"
prompt_filename = f"dataset/{problem}_prompt.txt"

In [31]:
def main():
  system_msg = ""
  if   lang == "verilog":  system_msg = verilog_system_msg
  else: 
    print("")
    print(f"ERROR: Unknown language {lang}")
    print("")
    return
  
  problem = "?"
  if prompt_filename.endswith("_prompt.txt"):
    problem = os.path.basename(prompt_filename[:-11])
  
  print( "" )
  print( f"problem     = {problem}"     )
  print( f"model       = {model}"       )
  print( f"temperature = {temperature}" )
  print( f"top_p       = {top_p}"       )
  print( f"max_tokens  = {max_tokens}"  )

  # Read the prompt file

  with open(prompt_filename) as file:
    prompt = file.read()

  # Create full prompt (does NOT load any examples)

  full_prompt = ""
  full_prompt += "\nQuestion:\n"
  full_prompt += prompt.strip() + "\n"
  full_prompt = full_prompt.rstrip() + "\n" + prompt_no_explain_suffix
  full_prompt += "\nAnswer:\n"

  # Print system message and prompt

  print("")
  print("System Message")
  print("-"*74)
  print(system_msg)

  print("Prompt")
  print("-"*74)
  print(full_prompt.rstrip())

  # Create LLM messages

  msgs = [ SystemMessage(system_msg), HumanMessage(full_prompt) ]

  # Query the LLM

  llm = ChatOpenAI(
    model        = model,
    temperature  = temperature,
    top_p        = top_p,
    max_tokens   = max_tokens,
  )

  with get_openai_callback() as cb:
    resp = llm.invoke(msgs)

  # Display the response

  print("")
  print("Response")
  print("-"*74)

  print("")
  print(resp.content)
  print("")

  # Display statistics

  print("Statistics")
  print("-"*74)
  
  print("")
  print(f"prompt_tokens = {cb.prompt_tokens}")
  print(f"resp_tokens   = {cb.completion_tokens}")
  print(f"total_tokens  = {cb.total_tokens}")
  print(f"total_cost    = {cb.total_cost}")
  print("")

  # Extract code from response
  if output != "NONE":
    file = open( output, 'w' )

    print( "" , file=file)

    # Scan response for code using <CODE></CODE>

    found_code_lines = []
    found_code_start = False
    found_code_end   = False

    for line in iter(resp.content.splitlines()):

      if not found_code_start:
        if line.strip() == "<CODE>":
          found_code_start = True
        elif line.lstrip().startswith("<CODE>"):
          found_code_lines.append( line.lstrip().replace("<CODE>","") )
          found_code_start = True

      elif found_code_start and not found_code_end:
        if line.strip() == "</CODE>":
          found_code_end = True
        elif line.rstrip().endswith("</CODE>"):
          found_code_lines.append( line.rstrip().replace("</CODE>","") )
          found_code_end = True
        else:
          found_code_lines.append( line )

    if found_code_start and found_code_end:
      for line in found_code_lines:
        print( line , file=file )

    # If did not find code by looking for <CODE></CODE>, then scan response
    # for code using backticks

    if not found_code_start and not found_code_end:

      found_code_lines = []
      found_code_start = False
      found_code_end   = False

      for line in iter(resp.content.splitlines()):

        if not found_code_start:
          if line.lstrip().startswith("```"):
            found_code_start = True

        elif found_code_start and not found_code_end:
          if line.rstrip().endswith("```"):
            found_code_end = True
          else:
            found_code_lines.append( line )

      if found_code_start and found_code_end:
        for line in found_code_lines:
          print(line , file=file )

        # Print comment so we can track responses that did not use
        # <CODE></CODE> correctly

        print( "" , file=file )
        if lang == "verilog":
          comment_delim = "//"
        else:
          comment_delim = "#"
        print( comment_delim + " PYHDL-EVAL: response did not use <CODE></CODE> correctly" , file=file )

    print( "" , file=file )
    file.close()

In [32]:
f = open( generate_log, 'w' )
orig_stdout = sys.stdout
sys.stdout = f

main()

sys.stdout = orig_stdout
f.close()